# Contents
1.[Hearing Lecture Matters](#1)

2.[Lecture Heard Student VS Lecture Not Heard Student](#2)

3.[Probability Distribution of Lecture Heard Student VS Lecture Not Heard Student](#3)

4.[Probability Distribution of Lecture Heard More Student VS Lecture Heard Less Student](#4)

5.[Types of Lectures](#6)

6.[Conclusion](#5)

<a id='1'></a>
## 1. Hearing Lecture Matters
   **Listening lectures is always makes the student more knowledgeable then the student who skipped the lecture. We will check this hypothesis that the student who heard the lectures performed well in answering the question.**

In [ ]:
import  pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

train= pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                usecols=[1, 2, 3,4,5,7,8,9], dtype={'timestamp': 'int64', 
                                                     'user_id': 'int32' ,
                                                     'content_id': 'int16',
                                                     'content_type_id': 'int8',
                                                     'task_container_id':'int16',
                                                     'answered_correctly':'int8',
                                                     'prior_question_elapsed_time': 'float32',
                                                     'prior_question_had_explanation': 'boolean'}
              )

<a id='2'></a>
## 2.Lecture Heard Student VS Lecture Not Heard Student
**In total we have 393656 stdent and out of this there are 149606 student who listed lectures and answered the question.There are 244050 students who did not listed lectures and answered the question**

In [ ]:
lecture_cnt = train[train.content_type_id == True][['user_id','content_type_id']].groupby(['user_id'],as_index = False).agg(['sum']).reset_index()
lecture_cnt.columns = ["user_id","lecture_heard_count"]
user_list = lecture_cnt['user_id'].unique()
train_lect = train[train.content_type_id == True]
train = train[train.content_type_id == False]
train_lecture_not_heard = train[~train.user_id.isin(user_list)]
train_lecture_heard = train[train.user_id.isin(user_list)]
train_lecture_not_heard_unq = pd.DataFrame(train_lecture_not_heard['user_id'].unique())
train_lecture_not_heard_unq.columns = ['user_id']
train_lecture_not_heard_unq['lecture'] = 'Not Heard'
train_lecture_heard_unq = pd.DataFrame(train_lecture_heard['user_id'].unique())
train_lecture_heard_unq.columns = ['user_id']
train_lecture_heard_unq['lecture'] = 'Heard'
train_lecture = pd.concat([train_lecture_not_heard_unq,train_lecture_heard_unq],axis=0)

In [ ]:

# visualizing Lecture Heard Student VS Lecture Not Heard Student
plt.figure(figsize=(10,5))
ax = sns.countplot(x=train_lecture['lecture'], palette=['#f76f6f',"#0cdeed"])
ax.set_xlabel('Lecture Heard Student VS Lecture Not Heard Student',size=15)
plt.title("Number of Student : 393656",size=15)

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.0f}'.format(p.get_height()), (x.mean(), y), ha='center', va='bottom')

In [ ]:
train_lecture_not_heard_mean = train_lecture_not_heard[['user_id','answered_correctly']].groupby(['user_id'],as_index = False).agg(['mean']).reset_index()
train_lecture_not_heard_mean.columns= ['user_id','train_lecture_not_heard_mean']
train_lecture_heard_mean = train_lecture_heard[['user_id','answered_correctly']].groupby(['user_id'],as_index = False).agg(['mean']).reset_index()
train_lecture_heard_mean.columns = ['user_id','train_lecture_heard_mean']

<a id='3'></a>
## 3.Probability Distribution of Lecture Heard Student VS Lecture Not Heard Student
Let's plot the probability distribution of of both catrgories of student.

In [ ]:
plt.rcParams['figure.figsize'] = [15,10]
plt.rcParams['font.size'] = 14
sns.kdeplot(train_lecture_heard_mean.train_lecture_heard_mean, label="Lecture Heard", clip=[0,1])
plt.axvline(train_lecture_heard['answered_correctly'].mean(), color='blue')
sns.kdeplot(train_lecture_not_heard_mean.train_lecture_not_heard_mean, label="Lecture Not Heard", clip=[0,1])
plt.axvline(train_lecture_not_heard['answered_correctly'].mean(), color='orange')

# add text 
plt.text(train_lecture_heard_mean.train_lecture_heard_mean.mean()-.3, 3,
         f"Lecture Heard Student Mean  mean={round(train_lecture_heard['answered_correctly'].mean(), 2)}")

plt.text(train_lecture_not_heard_mean.train_lecture_not_heard_mean.mean()-.25, 2.5,
         f"Lecture Not Heard Student Mean mean={round(train_lecture_not_heard['answered_correctly'].mean(), 2)}")

plt.title("Lecture Heard Vs Lecture Not Heard")
plt.xlabel("average answer correctness per user")
plt.ylabel("pdf")

plt.legend()
plt.show()

The mean value of the student who listened the lecture and answered the question is around.67 and the mean value of the student who did not listened the lecture and answered is around .57.The distribution of student who listened the lecture and answered is taller vs the student who did not listened the lectures and answered is wider, both the distribution varies widely.  

In [ ]:
del train,train_lecture_not_heard,train_lecture_not_heard_mean
train_lecture_heard = pd.merge(train_lecture_heard,lecture_cnt,on=['user_id'], how="left")

<a id='4'></a>
## 4.Probability Distribution of Lecture Heard More Student VS Lecture Heard Less Student
Let's plot the probability distribution of of both categories of student who listened more and less lecture.Student who has lecture listened count as 1 will go in to the less lectured heard group and rest of the student will go in to the more lecture listened group 

In [ ]:
train_lecture_heard_less = train_lecture_heard[train_lecture_heard.lecture_heard_count <= 1.0]
train_lecture_heard_more = train_lecture_heard[train_lecture_heard.lecture_heard_count > 1.0]

train_lecture_heard_less_mean = train_lecture_heard_less[['user_id','answered_correctly']].groupby(['user_id'],as_index = False).agg(['mean']).reset_index()
train_lecture_heard_less_mean.columns= ['user_id','train_lecture_heard_less_mean']
train_lecture_heard_more_mean = train_lecture_heard_more[['user_id','answered_correctly']].groupby(['user_id'],as_index = False).agg(['mean']).reset_index()
train_lecture_heard_more_mean.columns = ['user_id','train_lecture_heard_more_mean']

In [ ]:
plt.rcParams['figure.figsize'] = [15,10]
plt.rcParams['font.size'] = 14
sns.kdeplot(train_lecture_heard_more_mean.train_lecture_heard_more_mean, label="Lecture Heard More", clip=[0,1])
plt.axvline(train_lecture_heard_more['answered_correctly'].mean(), color='blue')
sns.kdeplot(train_lecture_heard_less_mean.train_lecture_heard_less_mean, label="Lecture Heard Less", clip=[0,1])
plt.axvline(train_lecture_heard_less['answered_correctly'].mean(), color='orange')

# add text 
plt.text(train_lecture_heard_more_mean.train_lecture_heard_more_mean.mean()-.3, 3,
         f"Lecture Heard More Student Mean mean={round(train_lecture_heard_more['answered_correctly'].mean(), 2)}")

plt.text(train_lecture_heard_less_mean.train_lecture_heard_less_mean.mean()-.25, 2.5,
         f"Lecture Heard Less Student Mean  mean={round(train_lecture_heard_less['answered_correctly'].mean(), 2)}")

plt.title("Lecture Heard More Vs Lecture Heard Less")
plt.xlabel("average answer correctness per user")
plt.ylabel("pdf")

plt.legend()
plt.show()

Student who listened lecture more performs better.

<a id='6'></a>
## 5.Types of Lectures
Let's plot the types of lectures heard by student

In [ ]:
lecture_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv', usecols=[0,3])
lecture_df.columns = ['lecture_id','type_of']
train = pd.merge(train_lect,lecture_df, left_on = ['content_id'], right_on = ['lecture_id'], how = 'left')

plt.figure(figsize=(10,5))
ax = sns.countplot(x=train['type_of'],order = train['type_of'].value_counts().index, palette=['#f76f6f',"#0cdeed","#EDDE0C","#0CEDDE"])
ax.set_xlabel('Lecture Types',size=15)
plt.title("Types of Lecture",size=15)

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.0f}'.format(p.get_height()), (x.mean(), y), ha='center', va='bottom')

Below plot is for the type of lecture student is most interested. Most of the times student interested to understand the concept.

In [ ]:
lec_type_cnt = train[train.content_type_id == True][['user_id','type_of','timestamp']].groupby(['user_id','type_of'],as_index = False).agg(['count']).reset_index()
lec_type_cnt.columns = ['user_id','type_of','lecture_type_count']
lec_type_cnt.sort_values(['user_id','lecture_type_count'],inplace=True,  ascending=False)
lec_type_cnt = lec_type_cnt.groupby('user_id').first()

plt.figure(figsize=(10,5))
ax = sns.countplot(x=lec_type_cnt['type_of'],order = lec_type_cnt['type_of'].value_counts().index, palette=['#f76f6f',"#0cdeed","#EDDE0C","#0CEDDE"])
ax.set_xlabel('Lecture Types',size=15)
plt.title("Types of Lecture",size=15)

for p in ax.patches:
    x=p.get_bbox().get_points()[:,0]
    y=p.get_bbox().get_points()[1,1]
    ax.annotate('{:.0f}'.format(p.get_height()), (x.mean(), y), ha='center', va='bottom')

<a id='5'></a>
## 6.Conclusion
1. Students who heard lectures performed better answering question.
2. Students who did not listened lectures performed very worst.This is very evident from the left side tail of non heard lectures students distribution
3. Students who did not listened lectures also outperfomed students who listened lectures and answer. This was evident from the right side tail of non heard lectures student distribution. Those student may well in advance prepared by other sources than lectures and answered the question. 


*Utilize lecture heard in you model*